In [1]:
import pyspark
from pyspark.sql import SparkSession
from datetime import datetime
from collections import namedtuple
from pyspark.sql import types
import pandas as pd

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/25 17:32:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
start = datetime(year=2020, month=1, day=1)

In [5]:
def filter_outliners(row):
    return row.lpep_pickup_datetime >= start

In [6]:
def prepare_for_grouping(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)
    

In [7]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count

    return (output_amount, output_count)

In [8]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [9]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [10]:
df_tmp_result = rdd \
    .filter(filter_outliners) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema)

In [12]:
df_tmp_result.write.parquet('tmp/green-revenue', mode='overwrite')

24/02/25 17:35:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/02/25 17:35:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/02/25 17:35:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/02/25 17:35:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/02/25 17:35:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [15]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [16]:
def model_predict(df):
    return df.trip_distance * 5

In [17]:
def apply_model_in_batch(partition):
    df = pd.DataFrame(partition, columns=columns)
    df['predicted_duration'] = model_predict(df)

    for row in df.itertuples():
        yield row

In [18]:
df_predicts = duration_rdd.mapPartitions(apply_model_in_batch).toDF().drop('Index')

In [19]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
| 4.050000000000001|
|              3.75|
|              6.65|
|             12.95|
|              97.6|
|             52.45|
|               4.3|
|             23.35|
|               0.0|
|               3.5|
|               4.0|
|             19.55|
| 7.800000000000001|
|               9.3|
| 6.949999999999999|
|               0.0|
|             47.05|
|            -39.05|
|             12.85|
|              13.5|
+------------------+
only showing top 20 rows

